# 既存の計算結果を読み込んで実行する

以下の流れで、既存のaction ID(パラメータ)とその評価値を読み込み、PHYSBO を実行することができます。

1. 外部ファイルを読み込み、既存のaction ID(パラメータ)と評価値を読み込む。
2. action ID(パラメータ)と評価値をPHYSBOに登録する。
3. PHYSBO から次に実行するパラメータを得る。

時間制限の関係上、PHYSBO をずっと開いたままにできないため、インタラクティブに実行できないといった場合に、利用することができます。

## 探索候補データの準備

これまでのマニュアルでも利用していたs5-210.csv を用いてガウス過程を実行します。

In [1]:
import physbo

import numpy as np


def load_data():
    A =  np.asarray(np.loadtxt('data/s5-210.csv',skiprows=1, delimiter=',') )
    X = A[:,0:3]
    t  = -A[:,3]
    return X, t

X, t = load_data()
X = physbo.misc.centering(X)

## 事前に計算したデータの用意

上述の `load_data` 関数では全てのXとtが格納されています。ここでは事前に計算したとして、actoin IDのリストをランダムに20個取得し、その評価値を得ます。

In [2]:
import random
random.seed(0)
calculated_ids = random.sample(range(t.size), 20)
print(calculated_ids)
t_initial = t[calculated_ids]

[12623, 13781, 1326, 8484, 16753, 15922, 13268, 9938, 15617, 11732, 7157, 16537, 4563, 9235, 4579, 3107, 8208, 17451, 4815, 10162]


## action ID(パラメータ)と評価値をPHYSBOに登録

policyの初期変数　`initial_data` に `calculated_ids` と `t[calculated_ids]` をリストとして登録します。

In [3]:
# policy のセット 
policy = physbo.search.discrete.policy(test_X=X, initial_data=[calculated_ids, t_initial])

# シード値のセット 
policy.set_seed( 0 )

## PHYSBO から次に実行するパラメータを取得

ベイズ最適化を行い、次の候補点を得ます。

In [4]:
actions = policy.bayes_search(max_num_probes=1, simulator=None, score="TS", interval=0,  num_rand_basis = 5000)
print(actions, X[actions])

Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0 -th epoch marginal likelihood -20.09302189053099
50 -th epoch marginal likelihood -23.11964735598211
100 -th epoch marginal likelihood -24.83020118385076
150 -th epoch marginal likelihood -25.817906570042602
200 -th epoch marginal likelihood -26.42342027124426
250 -th epoch marginal likelihood -26.822598600211865
300 -th epoch marginal likelihood -27.10872736571494
350 -th epoch marginal likelihood -27.331572599126865
400 -th epoch marginal likelihood -27.517235815448124
450 -th epoch marginal likelihood -27.67892333553869
500 -th epoch marginal likelihood -27.82299469827059
Done

[73] [[-1.6680279  -1.46385011  1.68585446]]


得られた候補点について外部で計算を行い、ファイルにactionsとそのスコアを登録する。再びファイルを読み込み、ベイズ最適化を実行し次の候補点を得るというプロセスを繰り返すことで、ベイズ最適化を進めることができます。